# 0. Importing the Libraries

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, re, json, time
import itertools, collections
from importlib import reload
from IPython.display import display
import scipy.sparse
import nltk
from w266_common import utils, vocabulary, tf_embed_viz

# 1. Getting Tickers and Tweets

We have two different data folders ('data' and 'data2'). This is because we first got all the tweets that mentioned name of stock as per the convention $ (ticker). 

Afterwards we got tweets that include names of companies along with their ticker symbols in the tweets. For example, we gathered any tweet that mentioned Amazon, and any tweet that mention $ (AMZN). This substantially increased our tweet count.

Our tweets dataset consist of tweets pertaining to S&P500 Top 50 companies only.

In [13]:
# getting all the files in folder 'data'
files = []
for file in os.listdir('data'):
    if file.endswith('.csv'):
        files.append(os.path.join('data', file))

In [14]:
# creating dataframe from files in 'data' folder

data = []

for filename in files:
    with open(filename, 'r') as file:
        ticker = filename.split('_')[-1][:-4]
        line = file.readline()
        line = file.readline()
        while(line):
            ts =line.split(';"')[0][1:].split(';')[0]
            tweet = line.split(';"')[1].split('";')[0]
            tweet_id = line.split('/')[-1][:-1] 
            line=file.readline()
            data.append([tweet_id,ticker,ts,tweet])
        
#df = pd.DataFrame(data, columns=['tweet_id','ticker','timestamp','tweet'])
#df

In [15]:
# getting all the files in 'data2' folder
files1 = []
for file in os.listdir('data2'):
    if file.endswith('.csv'):
        files1.append(os.path.join('data2', file))

In [16]:
# creating dataframe for files in 'data2' folder


for filename in files1:
    with open(filename, 'r') as file:
        ticker = filename.split('_')[-1][:-4]
        line = file.readline()
        line = file.readline()
        while(line):
            ts =line.split(';"')[0][1:].split(';')[0]
            tweet = line.split(';"')[1].split('";')[0]
            tweet_id = line.split('/')[-1][:-1] 
            line=file.readline()
            data.append([tweet_id,ticker,ts,tweet])
        
df = pd.DataFrame(data, columns=['tweet_id','ticker','timestamp','tweet'])

As can be seen from shapes of dataframes below, our first dataframe has considerably less tweets than the second one.

In [17]:
print(df.shape)

(3144677, 4)


In [18]:
df.head()

,tweet_id,ticker,timestamp,tweet
0,1108873286585331713,INTC,2019-03-21 23:29,$ AAPL $ TRV $ MSFT $ HD $ DWDP $ INTC $ NKE $...
1,1108872350223736833,INTC,2019-03-21 23:25,$ INTC Intel - Q3 2019 Intel Corporation Earni...
2,1108872312034492417,INTC,2019-03-21 23:25,$ INTC # Intel # Intelligence # ImplementingCl...
3,1108872250177060865,INTC,2019-03-21 23:25,$ INTC Intel - Q2 2019 Intel Corporation Earni...
4,1108872157822636038,INTC,2019-03-21 23:24,$ INTC Intel - Q1 2019 Intel Corporation Earni...


In [19]:
companies_dict = {'Union':'UNP', 'Disney':'DIS', 'Nike':'NKE', 'Chevron':'CVX',
                  'Intel':'INTC', 'Bank':'BAC', 'Pepsi':'PEP', 'Amgen':'AMGN',
                  'AT&T':'T', 'Procter':'PG', 'Microsoft':'MSFT', 'Wells':'WFG',
                  'Walmart':'WMT', 'Citigroup':'C', 'Verizon':'VZ', 'Exxon-Mobil':'XOM',
                  'Apple':'AAPL', 'Mastercard':'MA', 'Merck':'MRK', 'Boeing':'BA', 
                  'Comcast':'CMCSA', 'Salesforce':'CRM', 'Home':'HD', 'Berkshire':'BRK',
                  'Cisco':'CSCO', 'ATT':'T', 'Dow':'DWDP', 'Coca-Cola':'KO', 'Visa':'V',
                  'Facebook':'FB', 'Johnson':'JNJ', 'Abbott':'ABBT', 'Broadcom':'AVGO',
                  '3M':'MMM', 'Pfizer':'PFE', 'Amazon':'AMZN', 'Honeywell':'HON', 'Adobe':'ADBE',
                  'Google':'GOOG', 'Netflix':'NFLX', 'Eli':'LLY', 'Phillips':'PM', 'United':'UNP',
                  'AbbVie':'ABBV', 'McDonald':'MCD', 'JP':'JPM', 'Paypal':'PYPL', 'Oracle':'ORCL', 
                  'Medtronic':'MDT'}

In [20]:
# Associate tickers with company names

for key, value in companies_dict.items():
    df.loc[df['ticker'] == key, 'ticker'] = value
    #combined_df.at[index, 'ticker'] = companies_dict[row['ticker']]

In [21]:
# Get Nasdaq Stock Symbols
os.system("curl --ftp-ssl anonymous:jupi@jupi.com "
          "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt "
          "> nasdaq.lst")

0

In [22]:
!head -43 nasdaq.lst

<!DOCTYPE html><html data-adblockkey="MFwwDQYJKoZIhvcNAQEBBQADSwAwSAJBANDrp2lz7AOmADaN8tA50LsWcjLFyQFcb/P2Txc58oYOeILb3vBw7J6f4pamkAQVSQuqYsKx3YzdUHCvbVZvFUsCAwEAAQ==_pxrolOu5yyzUS0zMNgm01mNbFq+3Njr33BJOzQ+JCg65FKwTyH/JswWy8wg/VocffrVzS1j7NXxyt+jJ2gFZsQ=="><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8"><title></title><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="description" content="See related links to what you are looking for."/></head><!--[if IE 6 ]><body class="ie6"><![endif]--><!--[if IE 7 ]><body class="ie7"><![endif]--><!--[if IE 8 ]><body class="ie8"><![endif]--><!--[if IE 9 ]><body class="ie9"><![endif]--><!--[if (gt IE 9)|!(IE)]> --><body><!--<![endif]--><script type="text/javascript">g_pb=(function(){var
DT=document,azy=location,DD=DT.createElement('script'),aAA=false,LU;DD.defer=true;DD.async=true;DD.src="//www.google.com/adsense/domains/caf.js";DD.onerror=function(){if(azy.search!=='?z'){azy.href='/?z';}};DD.onlo

In [13]:
!tail -10 nasdaq.lst

ZSAN|Zosano Pharma Corporation - Common Stock|S|N|N|100|N|N
ZUMZ|Zumiez Inc. - Common Stock|Q|N|N|100|N|N
ZVZZC|NASDAQ TEST STOCK Nextshares Test Security|G|Y|N|100||Y
ZVZZT|NASDAQ TEST STOCK|G|Y|N|100||N
ZWZZT|NASDAQ TEST STOCK|S|Y|N|100||N
ZXYZ.A|Nasdaq Symbology Test Common Stock|Q|Y|N|100||N
ZXZZT|NASDAQ TEST STOCK|G|Y|N|100||N
ZYNE|Zynerba Pharmaceuticals, Inc. - Common Stock|G|N|N|100|N|N
ZYXI|Zynex, Inc. - Common Stock|S|N|N|100|N|N
File Creation Time: 0408201903:03|||||||


In [14]:
!tail -n +43 nasdaq.lst | cat | sed '$d' | sed 's/|/ /g' > nasdaq.lst2

In [15]:
!echo; head nasdaq.lst2; echo "..."; tail nasdaq.lst2


AABA Altaba Inc. - Common Stock Q N N 100 N N
AAL American Airlines Group, Inc. - Common Stock Q N N 100 N N
AAME Atlantic American Corporation - Common Stock G N N 100 N N
AAOI Applied Optoelectronics, Inc. - Common Stock G N N 100 N N
AAON AAON, Inc. - Common Stock Q N N 100 N N
AAPL Apple Inc. - Common Stock Q N N 100 N N
AAWW Atlas Air Worldwide Holdings - Common Stock Q N N 100 N N
AAXJ iShares MSCI All Country Asia ex Japan Index Fund G N N 100 Y N
AAXN Axon Enterprise, Inc. - Common Stock Q N N 100 N N
ABCB Ameris Bancorp - Common Stock Q N N 100 N N
...
ZS Zscaler, Inc. - Common Stock Q N N 100 N N
ZSAN Zosano Pharma Corporation - Common Stock S N N 100 N N
ZUMZ Zumiez Inc. - Common Stock Q N N 100 N N
ZVZZC NASDAQ TEST STOCK Nextshares Test Security G Y N 100  Y
ZVZZT NASDAQ TEST STOCK G Y N 100  N
ZWZZT NASDAQ TEST STOCK S Y N 100  N
ZXYZ.A Nasdaq Symbology Test Common Stock Q Y N 100  N
ZXZZT NASDAQ TEST STOCK G Y N 100  N
ZYNE Zynerba Pharmaceuticals, Inc. - Common Stock G

In [16]:
!awk '{print $1}' nasdaq.lst2 > nasdaq.csv

In [23]:
tickers = pd.read_csv('nasdaq.csv', index_col=None, header=None)
tickers.columns = ['ticker']

In [24]:
# # Get NYSE Symbols
nyse = pd.read_csv('companylist.csv')
cols = [1,2,3,4,5,6,7,8,9]
nyse.drop(nyse.columns[cols],axis=1,inplace=True)
nyse.columns = ['ticker']
tickers = tickers.append(nyse)
tickers = list(tickers['ticker'])

In [25]:
# Get Tweets associated with more than one stock...

# First, using the tweet id...
g = df.groupby('tweet_id').size().reset_index(name='count')
print('Tweets associated with more than one stock:',"{:,}".format(sum(g[g['count']>1]['count'])))

# Delete those tweet ids
tweets_ids_to_drop = list(g[g['count']>1]['tweet_id'])
df = df[~df.tweet_id.isin(tweets_ids_to_drop)]
df.shape

Tweets associated with more than one stock: 542,968


(2601709, 4)

In [26]:
# This one takes a while when run for the first time

import glove_helper; reload(glove_helper)

hands = glove_helper.Hands(ndim=100)

Loading vectors from data/glove/glove.6B.zip
Parsing file: data/glove/glove.6B.zip:glove.6B.100d.txt
Found 400,000 words.
Parsing vectors... Done! (W.shape = (400003, 100))


In [27]:
# Second, discard tweets that mention multiple stocks
# This one takes a while, hang in there
# Take advantage of the whole iteration and transform tweet for classifier
df_duplicate = df.copy()

TOKEN_RE = re.compile(r"\w.*?\b")
indices_to_drop = []

for index, row in df_duplicate.iterrows():
    
    print(index,end='\r')

    tweet_no_url = re.sub(r'http\S+', '', row['tweet'])
    tokens = tweet_no_url.split(' ')
    sentence = '<s> '
    flag = True
    prev_token = ''
    for token in tokens:
        try:
            a = hands.vocab.word_to_id[token.lower()]
            sentence = sentence + token.lower() + ' '

        except:
            pass

        if token in tickers and prev_token == '$':

            if token != row['ticker']:
                indices_to_drop.append(index)
                flag=False
                break
        
        prev_token = token
    
    sentence += '</s>'
    
    if flag:
        
        df_duplicate.at[index,'tweet'] = re.sub('[^A-Za-z0-9 <>/]+', '', sentence)

df_duplicate.drop(indices_to_drop, inplace=True)

In [31]:
df_duplicate.to_csv('../w266_final/clean_tweets.csv', index=False)

In [32]:
df_duplicate.head()

,tweet_id,ticker,timestamp,tweet
200,1105519541147262976,C,2019-03-12 17:22,<s> c the only big bank i buy </s>
201,1105519387367227393,C,2019-03-12 17:22,<s> on the density of </s>
203,1105517203074371584,C,2019-03-12 17:13,<s> ucla is the of c is so good at spinning a...
290,1105288514155929601,C,2019-03-12 02:04,<s> < </s>
323,1104039312251858945,DWDP,2019-03-08 15:20,<s> in inc after decline in shorted </s>
